In [22]:
import pandas as pd
import numpy as np
from scipy import stats

### 1
Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

water.exe

In [4]:
df = pd.read_csv("water.txt", sep="\t")

In [19]:
df.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [21]:
cols = df.columns[2:]
round(df[cols].corr().iloc[0, 1], 4)

-0.6548

### 2
В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

In [25]:
round(stats.spearmanr(df.mortality, df.hardness)[0], 4)

-0.6317

### 3
Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [35]:
res = [df[df.location == name][cols].corr().iloc[0, 1] for name in ("South", "North")]
res

[-0.6021532715484159, -0.36859783832887183]

In [38]:
round(
    min(
        np.abs(res)
        ),
    4
)

0.3686

### 4
Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

| X_1 \ X_2 | 0 | 1 |
| ---- | --- | --- |
| 0 | a | b |
| 1 | c | d|

| $ $ | женщины | мужчины |
| ---- | --- | --- |
| 1 раз в месяц | 203 | 239 |
| реже 1 раза в мес | 718 | 515 |

In [64]:
a = 203
b = 239
c = 718
d = 515

res = (a * d - b * c) / ((a+b)*(a+c)*(b+d)*(c+d))**0.5
round(res, 3)

-0.109

### 5
В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры (например, если вы получили $ 5.5\times10^{-8}$, нужно ввести 8).

In [61]:
res = stats.chi2_contingency([[a, b], [c, d]])[1]
abs(int(round(np.log10(res)))), res

(5, 1.0558987006638725e-05)

### 6
В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

  $ $ | $X_1$ | $X_2$  | 
  -------| ------ | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$


In [79]:
women = a
men = b
n1 = a + c
n2 = b + d

p1, p2 = women / n_1, men / n_2

In [82]:
def proportions_diff_confint_ind(p1, n1, p2, n2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [85]:
res = proportions_diff_confint_ind(p2, n2, p1, n1)
res

(0.053905233215813156, 0.13922183141523897)

In [86]:
round(res[0], 4)

0.0539

### 7
Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).


$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

In [87]:
def z_stat(p1, n1, p2, n2):
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [90]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [103]:
proportions_diff_z_test(z_stat(p1, n1, p2, n2))

8.153453089576601e-06

In [104]:
6

6

### 8
Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

|$ $ | Не доволен |Более или менее | Доволен|
| --- | --- | --- | --- |
Не очень счастлив |	197 |	111 | 33|
Достаточно счастлив | 382 | 685 | 331 |
Очень счастлив | 110 | 342 | 333 |


Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

Для решения этой задачи существует 2 способа.
- Первый способ — создать таблицу сопряженности и подсчитать по формуле из видео “Значимость корреляции”. Обратите внимание на то, что единица в формуле вычитается из всей суммы, а не из каждого слагаемого.


- Второй способ заключается в использовании функции stats.chi2_contingency, который вернет в качестве первого аргумента значение хи-квадрат для переданной таблицы сопряженности.

In [114]:
array = np.array(
    [[197, 111, 33],
     [382, 685, 331],
     [110, 342, 333]])
res = stats.chi2_contingency(array)
res

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))

In [113]:
round(res[0], 4)

293.6831

### 9
На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили $ 5.5\times10^{-8}$, нужно ввести 8)

In [110]:
res[1]

2.4964299580093467e-62

In [111]:
62

62

### 10
Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

In [121]:
res = np.sqrt(res[0] / (array.sum() * (min(array.shape) - 1)))
res

0.2412013934500338

In [122]:
round(res, 4)

0.2412